In [1]:
class ChlArctic:
    '''
    class ChlArctic(runname,resultpath,savepath,meshpath,dataset, observation_file,first_year,last_year,
                 savefig=False, verbose=False, output=False, plotting=True, Taylor=True)
                 
    This routine compare FESOM Chlorophyll outputs with state-of-the-art Arctic adapted satellite products:
    1) From OC-CCI: OCEANCOLOUR_ARC_BGC_L4_MY_009_124, https://doi.org/10.48670/moi-00293 => select dataset = 'OCCCI'
    2) From Stanford: Lewis and Arrigo, 2020. => select dataset = 'Lewis'
    
    self.ChlAfesom_surf_interp contains 2D dataset of 1x1 interpolated nanophytoplankton Chl.a
    self.DiaChlfesom_surf_interp contains 2D dataset of 1x1 interpolated diatom Chl.a
    self.Chl_total_interp = Chl_total contains 2D dataset of 1x1 interpolated sum of Chl.a
    self.unitfesom contains str of FESOM Chl.a unit
    '''
    
    def __init__(self,resultpath,savepath,mesh,ncfile,first_year,last_year,
                savefig=False,output=False,plotting=True,verbose=False,Taylor=True,runname='fesom'):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.fyear = first_year
        self.lyear = last_year
        self.savefig = savefig
        self.ncfile = ncfile
        self.verbose = verbose
        self.output = output
        self.plotting = plotting
        self.Taylor = Taylor

        import matplotlib.pyplot as plt
        import matplotlib.colors as colors
        import numpy as np
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        import scipy.io as spio
        import cartopy.feature as cfeature
        import pyfesom2 as pf
        from Py_f2recom_toolbox import plt_Taylor_norm
        import xarray as xr
        
        mapproj = pf.get_proj('np')

        if(self.verbose):
            print('Processing {0}'.format(self.resultpath))

        # resolution can be increased below if necessary
        londic, latdic = np.meshgrid(np.arange(-180,181,1), np.arange(65,90.5,0.5))
            
        # load OCCCI CHl.a data -------------------------------------------------------------------------------------
        Chlsat = xr.open_dataset(ncfile, decode_times=True)
        if 'LEWIS' in ncfile:
            tag = 'LEWIS'
            print('Lewis et al. dataset selected')
            label = 'Lewis et al. [2003-2021]'
            print(' !!! Only MODIS Satellite for the 2003-2021 period !!!')
            Chlsat = Chlsat.where(Chlsat.month.isin([5,6,7,8,9]), drop=True).mean(dim='month').compute()
            # interpolate Satellite Chl to the same regular grid -------------------------------------------------------------------------------------
            Chlsat_interp = griddata((Chlsat.longitude.data.ravel(), Chlsat.latitude.data.ravel()),
                                Chlsat.TChl.data.ravel(), (londic, latdic), method = 'nearest')
        elif 'OCCCI' in ncfile:
            tag = 'OCCCI'
            print('OCCCI. dataset selected')    
            label = 'OC-CCI [2000-2019]'
            print(' !!! Only Satellite data for the 2000-2019 period !!!')
            Chlsat = Chlsat.where(Chlsat.time.isin([5,6,7,8,9]), drop=True).mean(dim='time').compute()
            # interpolate Satellite Chl to the same regular grid -------------------------------------------------------------------------------------
            Chlsat_interp = griddata((Chlsat.longitude.data.ravel(), Chlsat.latitude.data.ravel()),
                                Chlsat.CHL.data.ravel(), (londic, latdic), method = 'nearest')
        else:
            print('wrong dataset selected, please select either {Lewis} or {OCCCI}')
                                   
        
        box=[-180, 180, 65, 90]
        unit = 'Chl.a [mg Chl m$^{-3}$]'          
        
        # load FESOM Chl.a data -------------------------------------------------------------------------------------        
        years = np.arange(self.fyear, self.lyear+1,1)
        fesom_label = 'FESOM-REcoM Chl.a {0}-{1}'.format(self.fyear,self.lyear)   
        
        PhyChlfesom = pf.get_data(self.resultpath, "PhyChl", years, mesh, 
                               how=None, compute=False, runid=self.runname, silent=True)
        DiaChlfesom = pf.get_data(self.resultpath, "DiaChl", years, mesh, 
                               how=None, compute=False, runid=self.runname, silent=True)

        PhyChlfesom = PhyChlfesom.where(PhyChlfesom.time.dt.month.isin([5,6,7,8,9]), drop=True).resample(time='YS').mean(dim='time').compute()
        DiaChlfesom = DiaChlfesom.where(DiaChlfesom.time.dt.month.isin([5,6,7,8,9]), drop=True).resample(time='YS').mean(dim='time').compute() 
        
        # load FESOM Cocco/Phaeo Chl.a data -------------------------------------------------------------------------------------------
        from pathlib import Path
        cocco_path = Path(self.resultpath + '/CoccoChl.fesom.'+str(years[0])+'.nc') # assuming that coccos were used for the entire simulation if they were used in the first year of simulation
        phaeo_path = Path(self.resultpath + '/PhaeoChl.fesom.'+str(years[0])+'.nc') # assuming that phaeo was used for the entire simulation if they were used in the first year of simulation
        
        
        if cocco_path.is_file():
            CoccoChlfesom = pf.get_data(self.resultpath, "CoccoChl", years, mesh, 
                               how=None, compute=False, runid=self.runname, silent=True)
            CoccoChlfesom = CoccoChlfesom.where(CoccoChlfesom.time.dt.month.isin([5,6,7,8,9]), drop=True).resample(time='YS').mean(dim='time').compute()

            if phaeo_path.is_file():
                print('4-phytoplankton model is used')
                PhaeoChlfesom = pf.get_data(self.resultpath, "PhaeoChl", years, mesh, 
                                   how=None, compute=False, runid=self.runname, silent=True)
                PhaeoChlfesom = PhaeoChlfesom.where(PhaeoChlfesom.time.dt.month.isin([5,6,7,8,9]), drop=True).resample(time='YS').mean(dim='time').compute()
                Chlfesom = np.nanmean(PhyChlfesom[:,:,0] + DiaChlfesom[:,:,0] + CoccoChlfesom[:,:,0] + PhaeoChlfesom[:,:,0], axis =0) # select only surface
            else:
                print('3-phytoplankton model is used')
                Chlfesom = np.nanmean(PhyChlfesom[:,:,0] + DiaChlfesom[:,:,0] + CoccoChlfesom[:,:,0], axis =0) # select only surface

        
        else:
            print('2-phytoplankton model is used')
            Chlfesom = np.nanmean(PhyChlfesom[:,:,0] + DiaChlfesom[:,:,0], axis =0) # select only surface
            
        # interpolate FESOM CHl.a to regular -------------------------------------------------------------------------------------
        Chlfesom_interp = pf.fesom2regular(
                data = Chlfesom,
                mesh = mesh,
                lons = londic, 
                lats = latdic)

        Chlfesom_interp_log10 = np.log10(Chlfesom_interp)

        # apply sea mask to OCCCI as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        Chlsat_ma = np.copy(Chlsat_interp)
        Chlsat_ma[~np.isfinite(Chlfesom_interp)] = np.nan
        Chlsat_ma_log10 = np.log10(Chlsat_ma)
        
        # check CHl.a data -------------------------------------------------------------------------------------
        if(self.verbose):
            print('\nChl.a\nOCCCI min = {2:5.4f}, max = {3:5.4f}\nFESOM min = {0:5.4f}, max = {1:5.4f}'.format(
                    np.nanmin(Chlfesom_interp),np.nanmax(Chlfesom_interp),
                    np.nanmin(Chlsat_ma),np.nanmax(Chlsat_ma)))

            print('\nlog10(Chl.a)\nOCCCI min = {2:5.4f}, max = {3:5.4f}\nFESOM min = {0:5.4f}, max = {1:5.4f}'.format(
                    np.nanmin(Chlfesom_interp_log10),np.nanmax(Chlfesom_interp_log10),
                    np.nanmin(Chlsat_ma_log10),np.nanmax(Chlsat_ma_log10)))
        
        if(self.plotting):
            # plot each CHl.a dataset -------------------------------------------------------------------------------------        
            
            levels = np.array([0,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,
                                   0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,
                                   1,2,3,4,5,7])
            ticks = [0,0.01,0.03,0.05,0.07,0.1,0.3,0.5,0.7,1,3,5,7]
            ticks_label = ['0','0.01','0.03','0.05','0.07','0.1','0.3','0.5','0.7','1','3','5','7'] # +7 ?

            def mygrid(m):
                m.add_feature(cfeature.LAND, zorder=1, edgecolor='none', facecolor='gray')
                
            fig, axes = plt.subplots(1,3, 
                                     subplot_kw=dict(projection=mapproj),
                                     gridspec_kw={'hspace': 0.01, 'wspace': 0.1},
                                     figsize=(20,7), constrained_layout=False)     

            # REcoM
            m1 = axes[0]
            f1 = m1.pcolormesh(londic, latdic, Chlfesom_interp, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
                                   #vmin=1e-3,vmax=5e3)
            mygrid(m1)
            m1.set_title(fesom_label, fontsize=16)


            # Satellite
            m2 = axes[1]
            f2 = m2.pcolormesh(londic, latdic, Chlsat_ma, 
                                   transform = ccrs.PlateCarree(),
                                   norm=colors.BoundaryNorm(boundaries=levels, ncolors=256))
            mygrid(m2)
            m2.set_title(label, fontsize=16)

            # add one colorbar for first row plots below figure
            cbar = fig.colorbar(f1,
                                ax = axes[:2], 
                                location ='bottom',
                                extend = 'max',
                                ticks = ticks,
                                fraction=0.046, pad=0.04) 
            #cbar.ax.tick_params(labelsize=14)
            cbar.ax.set_xticklabels(ticks_label, fontsize=16) 
            cbar.set_label('Chl.a [mg m$^{-3}$]', fontsize=16)
            cbar.ax.set_xticklabels(cbar.ax.get_xticklabels(), rotation=45)
                
            # REcoM - Satellite
            levels_diff = np.arange(-3,3,0.125)
            m3 = axes[2]
            f3 = m3.pcolormesh(londic, latdic, Chlfesom_interp - Chlsat_ma, 
                                   transform = ccrs.PlateCarree(),cmap = 'RdBu_r',
                                   norm=colors.BoundaryNorm(boundaries=levels_diff, ncolors=256))
            mygrid(m3)
            m3.set_title('FESOM-REcoM - '+label, fontsize=16)

            # add one colorbar for difference plot below figure
            cbar = fig.colorbar(f3,
                            ax = axes[2], 
                            orientation = 'horizontal',
                            #location ='bottom',
                            ticks = [-3,-2,-1,0,1,2,3],
                            extend = 'both',
                            fraction=0.046, pad=0.04) 
            cbar.ax.tick_params(labelsize=14)
            cbar.set_label('Chl.a [mg m$^{-3}$]', fontsize=16)
            cbar.ax.set_xticklabels(cbar.ax.get_xticklabels(), rotation=45)
                
            m1.text(-0.12, 1.05, 'A', transform=m1.transAxes,
                        size=30, weight='bold')
            m2.text(-0.12, 1.05, 'B', transform=m2.transAxes,
                            size=30, weight='bold')
            m3.text(-0.12, 1.05, 'C', transform=m3.transAxes,
                            size=30, weight='bold')
                    
            m1.set_extent(box, ccrs.PlateCarree())
            m2.set_extent(box, ccrs.PlateCarree())
            m3.set_extent(box, ccrs.PlateCarree())

            
            #plt.show(block=False) 
            # fig export  -------------------------------------------------------------------------------------
            if(self.savefig==True):
                plt.savefig(self.savepath+self.runname+'_'+'ArcChla_'+tag+'_'+str(years[0])+'to'+str(years[-1])+'.png', 
                        dpi = 300, bbox_inches='tight')
                plt.savefig(self.savepath+self.runname+'_'+'ArcChla_'+tag+'_'+str(years[0])+'to'+str(years[-1])+'.pdf', 
                        bbox_inches='tight')
            plt.show(block=False)  

        if(self.Taylor):
            # statistics  -------------------------------------------------------------------------------------            
            # preparation of datasets
            if np.isnan(np.min(Chlsat_ma_log10)): print('WARNING: Satellite field contains NaNs')
            if np.isnan(np.min(Chlfesom_interp_log10)): print('WARNING: FESOM field contains NaNs')

            # get statistics only from valid OCCCI gridpoints 
            ind_stat = np.where(np.isfinite(Chlsat_ma_log10))

            title = 'log10 surface Chlorophyll'
            print('\nStatistics for '+title)
            plt_Taylor_norm(Chlsat_ma_log10[ind_stat],Chlfesom_interp_log10[ind_stat],
                                    mask=True,title=title)

            
            #plt.show(block=False) 
            # fig export  -------------------------------------------------------------------------------------
            if(self.savefig==True):
                plt.savefig(self.savepath+self.runname+'_'+'ArcChla_'+tag+'_'+str(years[0])+'to'+str(years[-1])+'_Taylor.png', 
                        dpi = 300, bbox_inches='tight')
                plt.savefig(self.savepath+self.runname+'_'+'ArcChla_'+tag+'_'+str(years[0])+'to'+str(years[-1])+'_Taylor.pdf', 
                        bbox_inches='tight')
            plt.show(block=False)  
        
        if(self.output):
            self.lon = londic
            self.lat = latdic
            self.chl_oc = Chlsat_ma
            self.chl_fesom = Chlfesom_interp
            self.unit = unitfesom